In [18]:
from time import sleep
import requests
import pandas as pd
# import matplotlib.pyplot as plt
from config import api_key
import datetime as dt
import json
from pprint import pprint
import pymongo
from pymongo import MongoClient

In [75]:

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [76]:
db = client["stocktests2"]

In [86]:
collection = db["stock_collection2"]

In [ ]:
#create a function to generate a list of all tickers
sp_500 = pd.read_csv("../Datasets/constituents_csv.csv")
sp_500.count()
ticker_symbol = sp_500['Symbol']



In [16]:
ticker_list =[]
for x in range(len(sp_500)):
#     sector_name = sp_500['Sector'][x]
#     sector_id = session.query(Sector.sector_id).filter(Sector.sector == sector_name).one()[0]
    ticker_list.append(sp_500['Symbol'][x])


In [70]:
ticker_list
ticker_list_test = ticker_list[0:9]
ticker_list_test

['AMZN', 'BBY', 'KMX', 'DISCA', 'EXPE', 'GRMN', 'M', 'MAR', 'MCD']

In [31]:
base_url = "https://www.alphavantage.co/query?"
def alpha_stock_request(function,symbol):
    return requests.get(base_url + f"function={function}&symbol={symbol}&outputsize=full&apikey={api_key}")

# def alpha_currency_request(function,from_symbol,to_symbol):
#     return requests.get(base_url + f"function={function}&from_symbol={from_symbol}&to_symbol={to_symbol}&apikey={api_key}")

In [82]:
# ticker_list = db.stocksTest.find()
# ticker_lista = [x[0] for x in ticker_list]
list_of_dicts = []

for ticker in ticker_list_test:

        sleep(3)
        try:
            results = alpha_stock_request("TIME_SERIES_DAILY",ticker).json()
            today = results['Meta Data']['3. Last Refreshed']
            data = results['Time Series (Daily)']

            for x in data:
                if x == today: #get yesterday and before data (exclude today)
                    year , month , day = x.split("-")
                    date_ref = year + month + day
                    row = data[x]
#                     row2 = ticker_list[y]
                    open_price = row['1. open']
                    close_price = row['4. close']
                    high_price = row['2. high']
                    low_price = row['3. low']
                    volume = row['5. volume']
                    
                    list_of_dicts.append(
                        {
                        "ticker": ticker,
                        "date": date_ref,
                         "open_price" : open_price,
                         "close_price" : close_price,
                         "high_price" : high_price,
                         "low_price" : low_price,
                         "volume" : volume
                      }
                    )
   

    #create mongo ORM
    #                 new_stock = Stock(ticker = ticker,
    #                                   date_id = date_ref,
    #                                   open_price = open_price,
    #                                   close_price = close_price,
    #                                   high_price = high_price,
    #                                   low_price = low_price,
    #                                   low_price = low_price,)
    #                 session.add(new_stock)
    #                 session.commit()
#                     print(data)


        except KeyError:
            print(f"Company {ticker} Not Found")

In [91]:
def insert_stocks(list_of_dicts):
#     db.stock_collection.drop()
    document = collection.insert_many(list_of_dicts)
#     document.acknowledged

In [93]:
iddd = insert_stocks(list_of_dicts)

In [95]:
print(iddd)

None


In [92]:
client.close()

In [90]:
list_of_dicts



[{'ticker': 'AMZN',
  'date': '20200402',
  'open_price': '1901.6400',
  'close_price': '1918.8300',
  'high_price': '1927.5300',
  'low_price': '1890.0000',
  'volume': '4305149'},
 {'ticker': 'BBY',
  'date': '20200402',
  'open_price': '53.1300',
  'close_price': '52.7600',
  'high_price': '55.0601',
  'low_price': '51.7110',
  'volume': '3328242'},
 {'ticker': 'KMX',
  'date': '20200402',
  'open_price': '49.0100',
  'close_price': '50.2000',
  'high_price': '52.8300',
  'low_price': '48.2600',
  'volume': '4687580'},
 {'ticker': 'DISCA',
  'date': '20200402',
  'open_price': '19.1800',
  'close_price': '18.6800',
  'high_price': '20.0450',
  'low_price': '18.4600',
  'volume': '5779817'},
 {'ticker': 'EXPE',
  'date': '20200402',
  'open_price': '51.0000',
  'close_price': '47.8600',
  'high_price': '54.4900',
  'low_price': '46.8000',
  'volume': '4978783'},
 {'ticker': 'GRMN',
  'date': '20200402',
  'open_price': '70.9100',
  'close_price': '73.4600',
  'high_price': '73.6600',